# Facemask Classifier

This notebook serves as a basic showcase of this project and the results we achieved.

> **Note:** we assume that at this point you have already set up a virtual environment as outlined in the [README](./README.md) and that all necessary dependencies are installed.

First things first, get imports and setup out of the way.

In [ ]:
import torch
import sys
import os
import random
import matplotlib.pyplot as plt
from torchvision.transforms.functional import to_pil_image
import numpy as np

In [ ]:
# for the sake of consistency and reproducability in this demo, we set a fixed seed. But feel free to play around or omit this part.
seed=1337
torch.manual_seed(seed)
random.seed(seed)

Let's download the evaluation dataset. Note, the model was **not** trained on these images, although they stem from the same dataset. Refer to the project's [README](./README.md) for the original dataset source.

In [ ]:
from util import download_dataset, dataset_url
dataset_path='dataset'
download_dataset(dataset_url, out_path=dataset_path)

Now for the pre-trained model.

In [ ]:
from util import download_model, model_url
model_path='model.pt'
download_model(model_url, out_path=model_path)

Let's choose a small subsample of images from the dataset and run our model against it.

In [ ]:
from architecture import Model1
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Subset
from torchvision.datasets import ImageFolder

model = Model1()
model.load_state_dict(torch.load(model_path))
model.eval()

testset_path = os.path.join(dataset_path, 'test')

# collect data for displaying later
eval_results=[]

test_imgs = ImageFolder(
        root=testset_path,
        transform=transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Resize((256, 256), antialias=True),
            ]
        ),
)

# try to use a perfect square, makes plotting calculations nicer :)
num_samples=16
test_set = Subset(test_imgs, random.sample(range(0, len(test_imgs)-1), num_samples))
test_loader = DataLoader(test_set, shuffle=True)

def get_class(idx):
    return test_set.dataset.classes[idx]

num_correct = 0
for data, label in test_loader:
    actual = label.item()
    prediction = 1 if torch.sigmoid(model(data)).item() > 0.5 else 0
    print(
        f"Actual: {actual} ({get_class(actual)}), Prediction: {prediction} ({get_class(prediction)})"
    )
    eval_results.append( (data.squeeze(), prediction, actual) )
    if actual == prediction:
        num_correct += 1
print(f"Accuracy: {num_correct/num_samples}")

In [ ]:
plt_samples=[test_set.dataset.imgs[x] for x in test_set.indices]
axis = int(np.ceil(np.sqrt(num_samples)))
fig, ax = plt.subplots(axis, axis, figsize = (16, 16))
i = 0
for row in range(axis):
     ax_row = ax[row]
     for column in range(axis):
         ax_column = ax_row[column]
         ax_column.set_xticklabels([])
         ax_column.set_yticklabels([])
         img = to_pil_image(eval_results[i][0])
         ax_column.imshow(img, cmap='gray')
         actual = eval_results[i][2]
         predicted = eval_results[i][1]
         is_correct = actual == predicted
         col = 'blue'
         ax_column.set_title(f"actual: {get_class(actual)}\npredicted: {get_class(predicted)}",
                    color = 'green' if is_correct else 'red')
         i += 1